In [1]:
%env PYTORCH_ENABLE_MPS_FALLBACK=1

import torch
from torch.utils.data import Dataset, DataLoader

from tqdm.notebook import tqdm
import wandb

import pandas as pd

wandb.login()

env: PYTORCH_ENABLE_MPS_FALLBACK=1


wandb: Currently logged in as: bhavye-mathur. Use `wandb login --relogin` to force relogin


True

In [2]:
DEVICE = "cpu"

In [3]:
def get_dense_model(input_size: int,
                    hidden_sizes: list[int],
                    output_size: int,
                    activation_func: callable):
    layers = []

    for size in hidden_sizes:
        layers.append(torch.nn.Linear(input_size, size))
        layers.append(activation_func())
        input_size = size

    layers.append(torch.nn.Linear(input_size, output_size))

    return torch.nn.Sequential(*layers)

In [4]:
means = {"U": 5.589, "V": 0.018}
stds = {"U": 9.832, "V": 3.232}

ESTIMATE_QUANTILE = 0.9935
N = 100000000

BATCH_SIZE = 2056
TRAIN_TEST_SPLIT = 0.8
TRAIN_VAL_SPLIT = 0.9

In [5]:
class WindDataset(Dataset):
    data: pd.DataFrame

    def __init__(self, variable: str, subset: str):
        if subset == "train":
            self.x = self.data.iloc[:int(len(self.data) * TRAIN_TEST_SPLIT)]
            self.x = self.data.iloc[:int(len(self.x) * TRAIN_VAL_SPLIT)]
        elif subset == "test":
            self.x = self.data.iloc[int(len(self.data) * TRAIN_TEST_SPLIT):]
        elif subset == "validation":
            self.x = self.data.iloc[:int(len(self.data) * TRAIN_TEST_SPLIT)]
            self.x = self.data.iloc[int(len(self.x) * TRAIN_VAL_SPLIT):]
        else:
            raise ValueError("Invalid Subset")

        self.y = self.x[variable]

        del self.x["U"]
        del self.x["V"]

        self.x = self.x.values.astype("float32")
        self.y = self.y.values.astype("float32")

        self.x = torch.Tensor(self.x).to(DEVICE)
        self.y = torch.Tensor(self.y).to(DEVICE)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, i):
        return self.x[i], self.y[i]

    @classmethod
    def init(cls, frac: float):
        cls.data = pd.read_feather(f"../raw/subset/UV-NGCT-{ESTIMATE_QUANTILE}-{100000000}.ft").sample(frac=frac)


In [6]:
VARIABLE = "U"

WindDataset.init(0.1)

train = WindDataset(VARIABLE, "train")
validation = WindDataset(VARIABLE, "validation")
test = WindDataset(VARIABLE, "test")

train_dl = DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
validation_dl = DataLoader(validation, batch_size=BATCH_SIZE, shuffle=False)
test_dl = DataLoader(test, batch_size=BATCH_SIZE, shuffle=False)

del WindDataset.data

In [7]:
INPUT_SIZE = 15
OUTPUT_SIZE = 1
HIDDEN_SIZES = [512, 256]
LEARNING_RATE = 1e-4

ACTIVATION = torch.nn.ReLU
LOSS_FUNC = torch.nn.MSELoss

In [8]:
model = get_dense_model(INPUT_SIZE, HIDDEN_SIZES, OUTPUT_SIZE, ACTIVATION)
model = model.to(DEVICE)
model


Sequential(
  (0): Linear(in_features=15, out_features=512, bias=True)
  (1): ReLU()
  (2): Linear(in_features=512, out_features=256, bias=True)
  (3): ReLU()
  (4): Linear(in_features=256, out_features=1, bias=True)
)

In [9]:
EPOCHS = 20

criterion = LOSS_FUNC()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)


In [14]:
wandb.init(project=f"MERRA2-{VARIABLE}", dir="wandb-local",
           config={"epochs": EPOCHS,
                   "learning_rate": LEARNING_RATE,
                   "batch_size": BATCH_SIZE,
                   "estimate_quantile": ESTIMATE_QUANTILE,
                   "layers": HIDDEN_SIZES,
                   "dataset": "NGCT"})

wandb.watch(model, log_freq=100)
model.train()


def evaluate_one_epoch(epoch):
    mse = 0
    mae = 0

    with torch.no_grad():
        for inputs, targets in validation_dl:
            prediction = model(inputs).squeeze()

            mse += torch.nn.functional.mse_loss(prediction, targets)
            mae += torch.nn.functional.l1_loss(prediction, targets)

    n = len(validation_dl)
    wandb.log({"val_rmse": ((mse / n) ** 0.5) * stds[VARIABLE]})
    wandb.log({"val_mae": (mae / n) * stds[VARIABLE]})


def train_one_batch(batch, batch_idx):
    optimizer.zero_grad()

    inputs, targets = batch

    prediction = model(inputs).squeeze()
    loss = criterion(prediction, targets)

    loss.backward()
    optimizer.step()

    if batch_idx % 100 == 0:
        with torch.no_grad():
            rmse = (torch.nn.functional.mse_loss(prediction, targets) ** 0.5) * stds[VARIABLE]
            wandb.log({"train_loss": loss})
            wandb.log({"train_rmse": rmse})


def train_one_epoch(epoch):
    data = iter(train_dl)
    for i in tqdm(range(len(train_dl))):
        train_one_batch(next(data), i)


def train():
    for epoch in tqdm(range(EPOCHS)):
        train_one_epoch(epoch)
        evaluate_one_epoch(epoch)


sweep_configuration = {
    "method": "bayes",
    "name": "sweep",
    "metric": {
        "goal": "minimize", 
        "name": "val_rmse"
	},
    "parameters": {
        "batch_size": {"values": [64, 128, 256, 512, 1024, 2048, 4096]},
        "learning_rate": {"max": 0.1, "min": 0.0001},
        "layers": {"values": []}
     }
}
# sweep_id = wandb.sweep(sweep=sweep_configuration, project=f"MERRA2-{VARIABLE}")
# wandb.agent(sweep_id, function=main, count=4)

train()

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/3502 [00:00<?, ?it/s]

  0%|          | 0/3502 [00:00<?, ?it/s]

  0%|          | 0/3502 [00:00<?, ?it/s]

  0%|          | 0/3502 [00:00<?, ?it/s]

  0%|          | 0/3502 [00:00<?, ?it/s]

In [ ]:
def test():
    model.eval()

    batches = len(test_dl)
    data = iter(test_dl)

    mse = 0
    mae = 0

    with torch.no_grad():
        for i in tqdm(range(batches)):
            inputs, targets = next(data)
            prediction = model(inputs).squeeze()

            mse += torch.nn.functional.mse_loss(prediction, targets)
            mse += torch.nn.functional.mse_loss(prediction, targets)

    return (mse / batches) ** 0.5 * stds[VARIABLE], (mae / batches) * stds[VARIABLE]


test_rmse, test_mae = test()


print(f"RMSE: {test_rmse} m/s")
print(f"MAE:  {test_mae} m/s")


In [13]:
wandb.finish()


train_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_rmse,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mae,█▅▃▂▁
val_rmse,█▆▃▂▁
train_loss,0.00209
train_rmse,0.44944
val_mae,0.11355
val_rmse,0.25638
